In [9]:
import pandas as pd
import urllib.request
import json
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
import numpy as np  
import folium


In [10]:
path = r"D:\OneDrive\energia_lab\data\\"

In [11]:
url = 'https://dadosabertos.aneel.gov.br/dataset/f9336ee6-7562-4c92-880a-d7217df6cc94/resource/28686698-544b-48e2-af58-ad4f7616246c/download/projetos-eficiencia-energetica-empresa.csv'
df = pd.read_csv(url, encoding='ISO-8859-1', sep=';', on_bad_lines='skip', header=0)

In [12]:
df['DatGeracaoConjuntoDados'] = pd.to_datetime(df['DatGeracaoConjuntoDados'])
df['DatInicioProjeto'] = pd.to_datetime(df['DatInicioProjeto'])
df['DatConclusaoProjeto'] = pd.to_datetime(df['DatConclusaoProjeto'])

In [13]:
df['VlrRetiradaDemandaPontaTotal'] = df['VlrRetiradaDemandaPontaTotal'].str.replace(',', '.').astype(float)
df['VlrRcbGlobal'] = df['VlrRcbGlobal'].str.replace(',', '.').astype(float)
df['VlrEnergiaEconomizadaTotal'] = df['VlrEnergiaEconomizadaTotal'].str.replace(',', '.').astype(float)
df['VlrCustoTotal'] = df['VlrCustoTotal'].str.replace(',', '.').astype(float)

In [14]:
# Define a small epsilon value to prevent division by zero
epsilon = 0.01

# Recalculate the score with epsilon added to the denominator to avoid division by zero
df['Score'] = (df['VlrEnergiaEconomizadaTotal'] + df['VlrRetiradaDemandaPontaTotal']) / (df['VlrCustoTotal'] + epsilon)

In [16]:
# Drop rows where 'Score' is null
df = df.dropna(subset=['Score'])

In [18]:
# Sort df by 'Score' in descending order
df = df.sort_values('Score', ascending=False)

In [38]:
# Calculate the 50th percentile of 'Score'
score_95th_percentile = df['Score'].quantile(0.95)

# Filter df to only include rows where 'Score' is above the 50th percentile
df_above_95 = df[df['Score'] > score_95th_percentile]

In [46]:
df = df_above_95

In [47]:
# Rescale 'Score' to range from 1 to 10
df['Score'] = 1 + (df['Score'] - df['Score'].min()) * (10 - 1) / (df['Score'].max() - df['Score'].min())

In [50]:
df['NomAgente'] = df['NomAgente'].str.upper()

In [52]:
# Instanciando o geocodificador com um timeout maior
geolocator = Nominatim(user_agent="unique_user_agent", timeout=10)

# Função para obter endereço e coordenadas com retry
def get_location(name, cache):
    retries = 3
    for _ in range(retries):
        if name in cache:
            return cache[name]
        try:
            location = geolocator.geocode(name)
            if location:
                result = (location.address, location.latitude, location.longitude)
            else:
                result = (None, None, None)
            cache[name] = result
            return result
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"Retry for {name} due to error: {str(e)}")
            time.sleep(5)  # Espera antes de tentar novamente
    return None, None, None

# Função para processar os nomes únicos e preencher as informações de localização
def process_names(names, delay=2):
    location_cache = {}
    results = {'address': [], 'latitude': [], 'longitude': []}
    for name in names:
        address, lat, lon = get_location(name, location_cache)
        results['address'].append(address)
        results['latitude'].append(lat)
        results['longitude'].append(lon)
        time.sleep(delay)  # Delay para respeitar os limites da API
    return results, location_cache

# Extrair nomes únicos de agentes
nomes_unicos = df['NomAgente'].unique()

# Processar os nomes para obter as informações de localização
location_results, cache = process_names(nomes_unicos)

# Adicionando os resultados ao DataFrame
df['Address'] = df['NomAgente'].map({name: addr for name, (addr, _, _) in cache.items()})
df['Latitude'] = df['NomAgente'].map({name: lat for name, (_, lat, _) in cache.items()})
df['Longitude'] = df['NomAgente'].map({name: lon for name, (_, _, lon) in cache.items()})

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212 entries, 5827 to 6037
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   DatGeracaoConjuntoDados       212 non-null    datetime64[ns]
 1   NomAgente                     212 non-null    object        
 2   IdeEmpresaProponenteProjeto   212 non-null    int64         
 3   DscCodProjeto                 212 non-null    object        
 4   DscTituloProjeto              212 non-null    object        
 5   DscTipologia                  212 non-null    object        
 6   VlrCustoTotal                 212 non-null    float64       
 7   VlrRcbGlobal                  211 non-null    float64       
 8   VlrEnergiaEconomizadaTotal    212 non-null    float64       
 9   VlrRetiradaDemandaPontaTotal  212 non-null    float64       
 10  DscObjetivo                   209 non-null    object        
 11  DscJustificativa              160

In [57]:
# Set pandas display options
pd.set_option('display.max_rows', None)

# Display value counts for 'NomAgente'
print(df['NomAgente'].value_counts(dropna=False))

NomAgente
COPEL DISTRIBUIÇÃO S.A                                         33
COMPANHIA PAULISTA DE FORÇA E LUZ                              16
CEMIG DISTRIBUIÇÃO S.A.                                        13
CELESC DISTRIBUIÇÃO S.A.                                       11
ENERGISA SUL-SUDESTE DISTRIBUIDORA DE ENERGIA S.A.              9
ELEKTRO REDES S.A.                                              8
COMPANHIA PIRATININGA DE FORÇA E LUZ                            8
RGE SUL DISTRIBUIDORA DE ENERGIA S.A.                           7
ENERGISA MATO GROSSO DISTRIBUIDORA DE ENERGIA S A               7
CEB DISTRIBUIÇÃO SA                                             6
CELESC DISTRIBUIÇÃO S.A                                         6
CENTRAIS ELETRICAS DO PARA                                      5
CENTRAIS ELÉTRICAS DE RONDÔNIA S.A   CERON                      4
COMPANHIA PAULISTA DE FORA E LUZ                                4
ENERGISA MATO GROSSO DO SUL - DISTRIBUIDORA DE ENERGIA S.A.     4


In [61]:
df['Address'].value_counts(dropna=False)

Address
None                                                                                                                                                                                                                                                                                       119
Copel Distribuição S.A., Rua Vicente Machado, Guarapuava, Região Geográfica Imediata de Guarapuava, Região Geográfica Intermediária de Guarapuava, Paraná, Região Sul, 85010-050, Brasil                                                                                                    33
Celesc Distribuição S.A., 2987, Jardim América, Chapecó, Região Geográfica Imediata de Chapecó, Região Geográfica Intermediária de Chapecó, Santa Catarina, Região Sul, 89803-903, Brasil                                                                                                   20
Companhia Paulista de Força e Luz, Avenida Sete de Setembro, Vila Pavani, Nova Granada, Região Imediata de São José do Rio Preto, R

In [62]:
# Count null values in 'Address', 'Latitude', and 'Longitude'
address_null_count = df['Address'].isnull().sum()
latitude_null_count = df['Latitude'].isnull().sum()
longitude_null_count = df['Longitude'].isnull().sum()

print(f"Null count in 'Address': {address_null_count}")
print(f"Null count in 'Latitude': {latitude_null_count}")
print(f"Null count in 'Longitude': {longitude_null_count}")

Null count in 'Address': 119
Null count in 'Latitude': 119
Null count in 'Longitude': 119


In [59]:
df.to_csv(path+'pee_filter_95.csv')